In [98]:
import matplotlib.pyplot as plt #imports
import numpy as np
from scipy.signal import butter, filtfilt, sosfiltfilt
from IPython.display import Audio, Image

In [99]:
def white(notes, durs, fs=44100, chorus=False, delay=0, reverbSpace='none', flanger=False, flangeRate=5, flangeDepth=.01, lowpass=False, FcLow = 0, highpass=False, FcHigh = 0, order=4):

  baseSound = noteCreate(notes, durs)

  ### Chorus (add major third to each note) ###
  finalSound = np.zeros((2, len(baseSound)))
  finalSound[0, :] = baseSound
  if chorus:
    finalSound[1, :] = noteCreate(np.array(notes)+4, durs)
    finalSound = np.sum(finalSound, axis=0)
  else:
    finalSound = finalSound.flatten()
  # end chorus


  ### Flanger (time varying delay) ###
  length = len(finalSound)
  currentSample = np.array(range(length))  
  lfo = 2+np.sin(2*np.pi*flangeRate*(currentSample)/fs)
  index = np.around(currentSample-fs*flangeDepth*lfo)
  index[index<0] = 0
  index[index>(length-1)] = length-1
  out = np.zeros(length)
  if flanger:
    for i in range(length):
        out[i] = np.float(finalSound[i]) + np.float(finalSound[int(index[i])])
    finalSound = out
    
  ### Delay (input range 0-44100, max corresponding to 1 second)
  h = np.zeros((1, fs)).flatten()
  h[0] = 1
  h[delay] = 1
  finalSound = np.convolve(finalSound, h)
  
  h1 = np.zeros((1, fs)).flatten()
  h1[0] = 1
  h1[delay] = 1
  finalSound = np.convolve(finalSound, h1)


  ### Convolution Reverb (choose from 3 environmental IR's) ### 
  if (reverbSpace=='hallway'):
    finalSound = convolution_reverb(finalSound,hallway)
  elif (reverbSpace=='garage'):
    finalSound = convolution_reverb(finalSound,garage)
  elif (reverbSpace=='deepspace'):
    finalSound = convolution_reverb(finalSound,deepspace)
    
  ### Lowpass and Highpass Filters ###
  if lowpass:
    finalSound = bandaid_low(finalSound, FcLow, order)

  if highpass:
    finalSound = bandaid_high(finalSound, FcHigh, order)

  return finalSound
  

In [4]:
def noteCreate(notes, durs):
    sr = 44100 #sample rate
    check = len(notes) #first check, same lengths
    check2 = len(durs)
    totalsound = np.array([])
    lengths = np.zeros(len(durs)) #zeros array to alter with lengths of notes in samples
    if check == check2: #ensures same inputs lengths
        count = check
        notes = newnote(notes)
        for i in range(count): #main iterater. 
            lengths[i] = sr * durs[i]
            whitey = np.random.randn(int(lengths[i])) #Generates a white noise for given number of samples
            sound = bandaid(data = whitey, cutoff = notes[i], order = 3) #run white noise through the steep band pass
            (r,e,soundEnv) = ADSR(sound,durs[i]) #applies ADSR to each note
            totalsound = np.concatenate((totalsound, soundEnv)) #concatenates the sounds
            
    else:
        print('please ensure that you have entered the same number of durations as notes') 
        return()
    finalSound = delayrace(totalsound, 65, 0.3)
    return(finalSound)

In [5]:
### This is for future use, if we want to add the ability to
### use note names instead of midi values.
### PLEASE IGNORE FOR NOW
def notecheck(notes):
#The sole purpose of this function to 
#convert any given note name from a list
#to its frequency equivalent. 
    freqs = np.zeros(len(notes))
    print(notes)
    print(freqs)
    for i in range(len(notes)):
        if notes[i] == "48" or "C":
            #np.put(freqs,i,130.81)
            freqs[i] = 130.81
        elif notes[i] == "49" or "C#" or "Db":  
            #np.put(freqs,i,138.59)
            freqs[i] = 138.59
        elif notes[i] == "50" or "D":
            np.put(freqs,i,146.83)
        elif notes[i] == "51" or "D#" or "Eb":
            np.put(freqs,i,155.56)
        elif notes[i] == "52" or "E":
            np.put(freqs,i,164.81)
        elif notes[i] == "53" or "F" or "E#":
            np.put(freqs,i,174.61)
        elif notes[i] == "54" or "F#" or "Gb":
            np.put(freqs,i,185.00)
        elif notes[i] == "55" or "G":
            np.put(freqs,i,196.01)
        elif notes[i] == "56" or "G#" or "Ab":
            np.put(freqs,i,207.65)
        elif notes[i] == "57" or "A":
            np.put(freqs,i,220.00)
        elif notes[i] == "58" or "A#" or "Bb":
            np.put(freqs,i,233.08)
        elif notes[i] == "59" or "B" or "Cb":
            np.put(freqs,i,246.94)
        elif notes[i] == "60" or "C2":
            np.put(freqs,i,261.63)
        elif notes[i] == "61" or "C#2" or "Db2":
            np.put(freqs,i,277.18)
        elif notes[i] == "62" or "D2":
            np.put(freqs,i,293.66)
        elif notes[i] == "56" or "D#2" or "Eb2":
            np.put(freqs,i,311.13)
        else:
            print('Please enter valid Note names (C - Eb2) or Midi numbers (48 - 63)')
            return();
    print(freqs)    
    return(freqs)

In [6]:
#This turns our midi values into frequencies
def newnote(notes):
    import numpy as np
    trueKeys = np.array(notes)
    relativeKeys = trueKeys-60
    freq = 440*2**(relativeKeys/12)
    return(freq)

In [7]:
#This applies a uniform ADSR envelope to each of our notes
def ADSR(x,dur, a=.2,d=.2,sustain=.5,r=.2,fs=44100):
    ti = fs * dur
    ax = np.arange(0,a,1/ti)
    ay = np.linspace(0,1,ax.size)
    
    if (sustain == 0): # sustain is false when the sustain level argument is 0
        rx = np.arange(a,a+r,1/ti)
        ry = np.linspace(1,0,rx.size)
        
        restx = np.arange(a+r,x.size/ti,1/ti)
        resty = np.linspace(0,0,restx.size)
        
        t = np.concatenate((ax,rx,restx))
        y = np.concatenate((ay,ry,resty))
        
        signal = np.multiply(x,y)
        return(t,y,signal);
    
    
    else:
        dx = np.arange(a,(a+d),1/ti)
        dy = np.linspace(1,sustain,dx.size)
    
        sx = np.arange(a+d,(x.size/ti)-r,1/ti)
        sy = np.linspace(sustain,sustain,sx.size)
    
        rx = np.arange((x.size/ti)-r,x.size/ti,1/ti)
        ry = np.linspace(sustain,0,rx.size)
    
        t = np.concatenate((ax,dx,sx,rx))
        y = np.concatenate((ay,dy,sy,ry))
    
        signal = np.multiply(x,y)
        return(t,y,signal);

In [8]:
#this is our main bandpass filter, which allows this synth to function

def butter_bandpass(cutoff1, cutoff2, order):
    fs = 44100
    nyq = 0.5 * fs
    normal_cutoff1 = cutoff1 / nyq
    normal_cutoff2 = cutoff2 / nyq
    white, off = butter(order, [normal_cutoff1, normal_cutoff2], btype='bandpass')
    return (white, off)

def bandaid(data, cutoff, order):
    fs = 44100
    cutoff1 = cutoff - 5
    cutoff2 = cutoff + 5
    white, off = butter_bandpass(cutoff1, cutoff2, order)
    song = filtfilt(white, off, data)#filtfilt
    return song

In [9]:
def delayrace(data,mm,scale):
    pad = np.zeros(int(len(data)/1000*mm))
    NewX = data/abs(data).max()
    song = np.concatenate([NewX,pad])
    d1song = np.concatenate([pad,NewX])
    final = song + (d1song * scale)
    return(final)

In [33]:
### Convolution Reverb ###

from scipy.io import wavfile
from scipy import signal
fs1, hallway = wavfile.read('./Conic Long Echo Hall.wav')
fs2, deepspace = wavfile.read('./Deep Space.wav')
fs3, garage = wavfile.read('./Parking Garage.wav')


def convolution_reverb(midi,IR):
    out = signal.convolve(midi,IR[:,1])
    return(out)

In [106]:
### Lowpass Filter ###

def butter_lowpass(cutoff, order):
    fs = 44100
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    low, off = butter(order, normal_cutoff, btype='low', output='sos') #svd filter
    return (low, off)

def bandaid_low(data, cutoff, order):
    fs = 44100
    low, off = butter_lowpass(cutoff, order)
    song = sosfiltfilt(low, data)
    return song

In [107]:
### Highpass Filter ###

def butter_highpass(cutoff, order):
    fs = 44100
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    high, off = butter(order, normal_cutoff, btype='hp', output='sos') # svd filter
    return (high, off)

def bandaid_high(data, cutoff, order):
    fs = 44100
    high, off = butter_lowpass(cutoff, order)
    song = sosfiltfilt(high, data)
    return song

In [121]:
## TEST ##

import numpy as np

#test1notes = ['C','D','E','F','G','A','B','C2']
test1notes = np.array([60,62,64,65,67,69,71,72])
test1len = np.ones(8)
test1len2 = [0.15,0.1,0.1,0.1,0.1,0.1,0.1,0.15]

test2notes = [50,52,53,55,52,48,50]
#test2ones = np.ones(7)
test2len = [0.25,0.25,0.25,0.25,0.5,0.25,1]
soundtest1 = white(test2notes,test2len, chorus=False, delay=0, reverbSpace='none', flanger=True, flangeRate=.75, flangeDepth=.01, lowpass = False, FcLow = 200, highpass = False, FcHigh = 10000, order = 3)
Audio(soundtest1,rate=44100)

In [12]:
## TEST2 ##
soundtest2 = white(test2notes,test2len, chorus=True, delay=10000)
Audio(soundtest2,rate=44100)